# Datagenerering til DigDirCamp 2020

av Bjørn Christian Weinbach, DigDir Campar.

## Bakgrunn

I vår applikasjon skal me demonstrere korleis tilgang til eins eigne persondata kan gjere det enklare å navigere i den norske offentlege sektor, spesielt for ungdom. Denne dataen er ikkje tilgjengeleg vha API i dag og vi er nøydd å generere denne dataen sjølv. Denne jupyter notebooken inneheld python-kode for generering av syntetiske data for ein populasjon som blei brukt som ein del av forarbeidet til prosjektet til DigDir Camparane i 2020.

## Naudsynte imports

In [ ]:
import pandas as pd
import numpy as np
import datetime
import random
import datetime

## Globale variablar

$$\text{testpnr} = \text{Test-personnummer}$$

In [ ]:
testpnr = ['23079412414',
           '23079412333',
           '23079412252',
           '23079412171',
           '31010650596',
           '27080350274',
           '04090751315',
           '18060550537',
           '23090350056',
           '23079418366',
           '23079418285',
           '23079418013',
           '23079417815']

## Globale Funksjonar

In [ ]:
def tlfnr():
    """Return phone no in the norwegian style."""
    return random.randint(10000000, 99999999)

def randomName(data):
    """Return random norwegian male or female name."""
    if random.randint(0, 1) == 1:   
        malename = np.array(data["Malename"].sample(1, replace = True))
        malelastname = np.array(data["Lastname"].sample(1, replace = True))
        return (malename + " " + malelastname)[0], (malename + "." + malelastname + "@norge.no")[0]
    else:
        femalename = np.array(data["Femalename"].sample(1, replace = True))
        femalelastname = np.array(data["Lastname"].sample(1, replace = True))
        return (femalename + " " + femalelastname)[0], (femalename + "." + femalelastname + "@norge.no")[0]

## Last inn datasett

### Kjelder

Norske navn: https://no.wikipedia.org/wiki/Liste_over_norske_mannsnavn , https://no.wikipedia.org/wiki/Liste_over_norske_kvinnenavn og https://no.wikipedia.org/wiki/Liste_over_norske_etternavn

Postnummer: https://data.norge.no/datasets/5e6847ba-156d-4e14-85d3-8d7f8b727523

In [ ]:
data = pd.read_csv('navn.csv')
post = pd.read_csv('postnr.csv', converters={'Postnummer': lambda x: str(x)}, delimiter=';')

## Kombiner til ein heil populasjon og lagre som JSON

In [ ]:
pnum = []
name = []
email = []
fnum = []
vig = []
adress = []
for t in testpnr:
    nam, ema = randomName(data)
    pnum.append(t)
    name.append(nam)
    email.append(ema)
    fnum.append(tlfnr())
    adress.append("Test adresse " + str(random.randint(1, 100)))

populasjon = pd.DataFrame()
populasjon['Personalno'] = pnum
populasjon['Name'] = name
populasjon['Email'] = email
populasjon['Phoneno'] = fnum
populasjon['Adress'] = adress
populasjon[post.columns] = pd.concat([post.sample(len(testpnr), replace = True, axis = 0)], axis=0, ignore_index=True)
populasjon.reset_index(drop=True)
populasjon = populasjon.set_index("Personalno")
populasjon.to_json(r'populasjon.json')
populasjon

## Kontaktinfo -> CSV for SQL import

In [ ]:
populasjon.to_csv('contact.csv')

## VIGO-Grades -> CSV for SQL import

In [ ]:
vigo_grades = pd.DataFrame()
vigo_grades['personalno'] = pnum
vigo_grades['RLE'] = np.random.randint(1, 6, len(testpnr))
vigo_grades['Naturfag'] = np.random.randint(1, 6, len(testpnr))
vigo_grades['Matematikk'] = np.random.randint(1, 6, len(testpnr))
vigo_grades = vigo_grades.reset_index(drop=True)
vigo_grades = vigo_grades.set_index("personalno")
vigo_grades.to_csv('vigo_grades.csv')

## VIGO-Absence -> CSV for SQL import

In [ ]:
vigo_absence = pd.DataFrame()
vigo_absence ['personalno'] = pnum
vigo_absence ['RLE'] = abs(np.random.normal(0.05, 0.05, len(testpnr)))
vigo_absence ['Naturfag'] = abs(np.random.normal(0.05, 0.05, len(testpnr)))
vigo_absence ['Matematikk'] = abs(np.random.normal(0.05, 0.05, len(testpnr)))
vigo_absence = vigo_absence.reset_index(drop=True)
vigo_absence = vigo_absence.set_index("personalno")
vigo_absence.to_csv('vigo_absence.csv')

## Helsenorge-fastlege -> CSV for SQL import

In [ ]:
docnam = []
for i in testpnr:
    tmp, _ = randomName(data)
    docnam.append(tmp)

healthgp = pd.DataFrame()
healthgp['personalno'] = pnum
healthgp['GP'] = docnam
healthgp = healthgp.reset_index(drop=True)
healthgp = healthgp.set_index("personalno")
healthgp.to_csv('healthgp.csv')